In [2]:
import os
from langchain_openai import ChatOpenAI
from getpass import getpass

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

chat = ChatOpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"],
    model='gpt-3.5-turbo-0125'
)

In [3]:
from langchain.schema import (
    SystemMessage,
    HumanMessage,
    AIMessage
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Hi AI, how are you today?"),
    AIMessage(content="I'm great thank you. How can I help you?"),
    HumanMessage(content="I'd like to understand string theory.")
]

In [4]:
res = chat(messages)


/tmp/ipykernel_39205/3952239659.py:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res = chat(messages)


In [5]:
print(res.content)

String theory is a theoretical framework in which the point-like particles of particle physics are replaced by one-dimensional objects called strings. These strings can vibrate at different frequencies, giving rise to different fundamental particles. The theory aims to provide a unified description of all fundamental forces of nature, including gravity.

There are different versions of string theory, such as superstring theory and M-theory, which incorporate supersymmetry and higher-dimensional objects beyond strings.

String theory has not yet been proven experimentally, but it is a widely studied and researched topic in theoretical physics due to its potential to unify all fundamental forces and explain the fundamental nature of the universe.

Is there anything specific you would like to know about string theory?


In [6]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="Why do physicists believe it can produce a 'unified theory'?"
)
# add to messages
messages.append(prompt)

# send to chat-gpt
res = chat(messages)

print(res.content)

Physicists believe that string theory has the potential to produce a unified theory because it attempts to describe all fundamental forces in a single framework. In traditional particle physics, there are different theories that explain different fundamental forces, such as the Standard Model of particle physics for electromagnetism, weak nuclear force, and strong nuclear force, and general relativity for gravity.

By introducing strings as the fundamental building blocks of the universe, string theory attempts to reconcile the principles of quantum mechanics and general relativity, which are currently described by separate theories. In doing so, it has the potential to provide a unified description of all fundamental forces and particles in the universe.

Additionally, string theory predicts the existence of additional dimensions beyond the familiar three spatial dimensions and one time dimension. These extra dimensions can help explain phenomena that are not accounted for in traditio

In [7]:
messages

[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi AI, how are you today?', additional_kwargs={}, response_metadata={}),
 AIMessage(content="I'm great thank you. How can I help you?", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="I'd like to understand string theory.", additional_kwargs={}, response_metadata={}),
 AIMessage(content='String theory is a theoretical framework in which the point-like particles of particle physics are replaced by one-dimensional objects called strings. These strings can vibrate at different frequencies, giving rise to different fundamental particles. The theory aims to provide a unified description of all fundamental forces of nature, including gravity.\n\nThere are different versions of string theory, such as superstring theory and M-theory, which incorporate supersymmetry and higher-dimensional objects beyond strings.\n\nString theory has not yet been proven e

### If you ask the chatGPT model about the latest model, for exmaple Deepseek R1, the model would fail to update anything after it has been trained.

In [6]:
# add latest AI response to messages
messages.append(res)

# now create a new user prompt
prompt = HumanMessage(
    content="What is so special about Deepseek R1?"
)
# add to messages
messages.append(prompt)

# send to OpenAI
res = chat(messages)

In [7]:
print(res.content)

I'm not familiar with Deepseek R1. Could you provide more context or information about what it is?


We can feed additional information to the LLM with some instructions to tell the model how we would like to use this information alogside our original questioin.

In [8]:
source_knowledge = (
    "We introduce our first-generation reasoning models, DeepSeek-R1-Zero and "
    "DeepSeek-R1. DeepSeek-R1-Zero, a model trained via large-scale "
    "reinforcement learning (RL) without supervised fine-tuning (SFT) as a "
    "preliminary step, demonstrates remarkable reasoning capabilities. Through "
    "RL, DeepSeek-R1-Zero naturally emerges with numerous powerful and "
    "intriguing reasoning behaviors. However, it encounters challenges such as "
    "poor readability, and language mixing. To address these issues and "
    "further enhance reasoning performance, we introduce DeepSeek-R1, which "
    "incorporates multi-stage training and cold-start data before RL. "
    "DeepSeek-R1 achieves performance comparable to OpenAI-o1-1217 on "
    "reasoning tasks. To support the research community, we open-source "
    "DeepSeek-R1-Zero, DeepSeek-R1, and six dense models (1.5B, 7B, 8B, 14B, "
    "32B, 70B) distilled from DeepSeek-R1 based on Qwen and Llama."
)

In [9]:
query = "What is so special about Deepseek R1?"

augmented_prompt = f"""Using the contexts below, answer the query.

Contexts:
{source_knowledge}

Query: {query}"""

In [10]:
# create a new user prompt
prompt = HumanMessage(
    content=augmented_prompt
)
# add to messages
messages.append(prompt)

# send to OpenAI
res = chat(messages)

In [11]:
print(res.content)

DeepSeek R1 is a special deep learning model that stands out for its impressive reasoning capabilities. It is designed to address challenges encountered by its predecessor, DeepSeek-R1-Zero, such as poor readability and language mixing, by incorporating multi-stage training and cold-start data before reinforcement learning (RL). This approach has led to DeepSeek R1 achieving performance comparable to OpenAI-o1-1217 on reasoning tasks. Additionally, the developers have open-sourced DeepSeek-R1-Zero, DeepSeek-R1, and six dense models distilled from DeepSeek-R1 to support the research community.


### Importing the Data

In this task, we will be importing our data. We will be using the Hugging Face Datasets library to load our data. Specifically, we will be using the "jamescalam/deepseek-r1-paper-chunked" dataset. This dataset contains the Deepseek R1 paper pre-processed into RAG-ready chunks.

In [12]:
from datasets import load_dataset

dataset = load_dataset(
    "jamescalam/deepseek-r1-paper-chunked",
    split="train"
)

dataset

/home/philippe/miniconda3/envs/rag/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 100%|██████████| 76/76 [00:00<00:00, 18999.11 examples/s]


Dataset({
    features: ['doi', 'chunk-id', 'chunk', 'num_tokens', 'pages', 'source'],
    num_rows: 76
})

In [ ]:
my_list: list[str] =  ["1", 2, "3"]
my_list2: list[int] = ["1", "3"]



[[[0, 0], [0, 0], [0, 0]],
 [[0, 0], [0, 0], [0, 0]],
 [[0, 0], [0, 0], [0, 0]],
 [[0, 0], [0, 0], [0, 0]],
 [[0, 0], [0, 0], [0, 0]]]

2